# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,uturoa,-16.7333,-151.4333,25.63,82,99,3.92,PF,1726524656
1,1,praia da vitoria,38.7333,-27.0667,23.20,88,100,6.37,PT,1726524657
2,2,edinburgh of the seven seas,-37.0676,-12.3116,10.26,67,26,17.28,SH,1726524658
3,3,blackmans bay,-43.0167,147.3167,9.29,67,97,0.45,AU,1726524659
4,4,georgetown,5.4112,100.3354,25.96,89,40,1.03,MY,1726524660


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo=True, 
    size="Humidity", 
    color="City", 
    alpha=0.5, 
    tiles="EsriWorldStreetMap", 
    frame_width=800, 
    frame_height=600
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[(city_data_df["Max Temp"] >= 20) & 
                               (city_data_df["Max Temp"] <= 30) & 
                               (city_data_df["Humidity"] < 50) & 
                               (city_data_df["Cloudiness"] < 10)]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,10,calama,-22.4667,-68.9333,22.27,5,0,10.29,CL,1726524667
65,65,ash shafa,21.0694,40.3119,22.23,36,2,0.38,SA,1726524730
75,75,port hedland,-20.3167,118.5667,23.03,25,0,5.14,AU,1726524741
196,196,jamestown,42.0970,-79.2353,25.80,41,0,3.09,US,1726524858
200,200,awjilah,29.1081,21.2869,27.84,42,0,3.80,LY,1726524888
290,290,colonia menno,-22.3667,-59.8167,25.80,30,0,3.54,PY,1726524996
327,327,hami,42.8000,93.4500,20.21,21,0,2.29,CN,1726525038
354,354,west richland,46.3043,-119.3614,25.42,35,0,2.06,US,1726524918
355,355,karratha,-20.7377,116.8463,21.76,32,0,5.31,AU,1726525071
358,358,presidencia de la plaza,-27.0015,-59.8424,20.71,36,0,2.83,AR,1726525074


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
10,calama,CL,-22.4667,-68.9333,5,
65,ash shafa,SA,21.0694,40.3119,36,
75,port hedland,AU,-20.3167,118.5667,25,
196,jamestown,US,42.0970,-79.2353,41,
200,awjilah,LY,29.1081,21.2869,42,
290,colonia menno,PY,-22.3667,-59.8167,30,
327,hami,CN,42.8000,93.4500,21,
354,west richland,US,46.3043,-119.3614,35,
355,karratha,AU,-20.7377,116.8463,32,
358,presidencia de la plaza,AR,-27.0015,-59.8424,36,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 1,  # We only want the first result (nearest hotel)
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
calama - nearest hotel: Apart Hotel Geotel Calama
ash shafa - nearest hotel: No hotel found
port hedland - nearest hotel: The Esplanade Hotel
jamestown - nearest hotel: DoubleTree Jamestown
awjilah - nearest hotel: No hotel found
colonia menno - nearest hotel: Hotel Mora
hami - nearest hotel: Хами
west richland - nearest hotel: Home2 Suites
karratha - nearest hotel: Karratha International Hotel
presidencia de la plaza - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
south hedland - nearest hotel: Port Haven
mafinga - nearest hotel: No hotel found
bir el hafey - nearest hotel: No hotel found
cuito - nearest hotel: No hotel found
zambezi - nearest hotel: No hotel found
nsanje - nearest hotel: No hotel found
copiapo - nearest hotel: Hotel Diego de Almeida
vernon - nearest hotel: Okanagan Royal Park Inn
korla - nearest hotel: Silver Star Hotel
maun - nearest hotel: Center Lodge Conference Center
santana do matos - nearest hotel: No hotel found

,City,Country,Lat,Lng,Humidity,Hotel Name
10,calama,CL,-22.4667,-68.9333,5,Apart Hotel Geotel Calama
65,ash shafa,SA,21.0694,40.3119,36,No hotel found
75,port hedland,AU,-20.3167,118.5667,25,The Esplanade Hotel
196,jamestown,US,42.0970,-79.2353,41,DoubleTree Jamestown
200,awjilah,LY,29.1081,21.2869,42,No hotel found
290,colonia menno,PY,-22.3667,-59.8167,30,Hotel Mora
327,hami,CN,42.8000,93.4500,21,Хами
354,west richland,US,46.3043,-119.3614,35,Home2 Suites
355,karratha,AU,-20.7377,116.8463,32,Karratha International Hotel
358,presidencia de la plaza,AR,-27.0015,-59.8424,36,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo=True, 
    size="Humidity", 
    color="City",
    alpha=0.5, 
    tiles="EsriWorldStreetMap", 
    frame_width=800, 
    frame_height=500,
    hover_cols=["City", "Hotel Name", "Country"],
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)